In [106]:
allure_etudier=1

In [107]:
import  sys

#LOCALISATION DES DONNEES
sys.path.insert(0, "C:/projets_python/diabolo")

import warnings
warnings.filterwarnings("ignore")



#LIBRAIRIES PERSO
import etude_variable.MyLog as log
#import etude_variable.jouer as jouer

#import etude_variable.analyse as ana

# LIBRAIRIE PYHTON CLASSIQUES
import pandas as pd

import numpy as np
import matplotlib
import scipy
import platform

from collections import Counter

#ESTIMATEUR
import sklearn
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import  RandomForestClassifier
from sklearn import neighbors, datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer

from sklearn.ensemble import GradientBoostingClassifier

#MISE A l ECHELLE
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import quantile_transform

from scipy.stats import uniform
from scipy.stats import randint

#TRAINING
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
import dask_searchcv as dcv
from skopt.space import Real, Integer
from sklearn.model_selection import cross_validate

#Evaluateur
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.feature_selection import SelectFromModel


from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.utils import class_weight

#outils
from dask.diagnostics import ProgressBar
import logging
from datetime import datetime


#Metriques
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn import  metrics
from sklearn.metrics import  roc_auc_score
from sklearn.metrics import  classification_report

#Outils

import time


#Graphique
from matplotlib import pyplot
from matplotlib import pyplot as plt
import seaborn as sns

print('Operating system version....', platform.platform())
print("Python version is........... %s.%s.%s" % sys.version_info[:3])
print('scikit-learn version is.....', sklearn.__version__)
print('pandas version is...........', pd.__version__)
print('numpy version is............', np.__version__)
print('matplotlib version is.......', matplotlib.__version__)
print('scipy version is.......', scipy.__version__)


Operating system version.... Windows-10-10.0.17134-SP0
Python version is........... 3.6.5
scikit-learn version is..... 0.19.1
pandas version is........... 0.22.0
numpy version is............ 1.14.2
matplotlib version is....... 2.2.0
scipy version is....... 1.0.0


# LECTURE DES DONNEES

In [108]:
# Creating bins for the win column
def assign_selection(W):
    if W <=2:
        return 1
    else:
        return 0
    
    # Creating bins for the win column
def assign_selection2(W):
    if W == 1.0:
        return 1
    if W == 0.0:
        return 0
    
def my_drop(df, col):
    if col in df:
        df.drop([col], axis=1, inplace=True)
    return df
    


def transformation(df2,allure_etudier):
    #Filtre des données¶
    print('\n\n----------- TRANSFORMATION --------- AJOUT DE COLONNE------------\n')
    start_time=timer()
    #df2=df2[df2.Point>0]
    df2 = df2[df2.PAR_COTEDER > 0]
    df2 = df2[df2.CO_PRIX > 0]
    df2 = df2[df2.PAR_ARRIVE > 0]  # On garde la ligne qui possede information arrivée
    df2 = df2.groupby("ALLURE")
    df2 = df2.get_group(allure_etudier)
    df2 = my_drop(df2, 'ALLURE')

    if allure_etudier == 2 or allure_etudier == 4 or allure_etudier == 5:
        df2 = df2[df2.POIDS > 20]
        df2 = df2[df2.POIDS < 80]
    #  on a lu les données avant l 'ajout de la colonne SELECTIOB
    df2['SELECTION2'] = df2['PAR_ARRIVE'].apply(assign_selection)
    df2 = my_drop(df2, 'PAR_ARRIVE')
  
    timer(start_time)
    return df2

In [109]:
def lecture_data(Fichier, xnames, xindex_col, allure=1, mode_debug=0, avec_index=True):
    
    print("DATA ....")
    start_time=timer()
    if avec_index==True:
        df = pd.read_csv(Fichier,  index_col=xindex_col,     sep=';',     names=xnames,               skipinitialspace=True,              encoding='Latin-1')
    else:
        df = pd.read_csv(Fichier,   index_col=None,  sep=';',                     names=xnames,                         skipinitialspace=True,     encoding='Latin-1')

    df = df.groupby("ALLURE")
    df = df.get_group(allure)
    #print(df.info())

    print(timer(start_time))
    return df


In [110]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        print(start_time)
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
        
        


# fit speciaux

In [111]:
  # Lecture des données sans index
xnames = ['ALLURE', 'CO_DISTANCE',
              'CO_PRIX', 'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE', 'PAR_ARRIVE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant',
              'PAR_PROPRIO',
              'NOM_JOC',
              'NOM_ENTR',
              'POIDS',
              'CORDE', 'CHEVAL',
              'MUSIC_CHEVAL',
              'MUSIC_ENT',
              'MUSIC_JOC',
              'PAR_VALEUR',
              'PAR_ENT_ECART_PLACE',
              'PAR_VICTOIRE',
              'PAR_VICTOIRE_Q',
          'PAR_ENT_NB_COURSE',
              'FIN_ligne']





    

In [112]:
def nettoyer_Nan(df):
    df[['PAR_ENT_ECART_GAGNANT']] = df[['PAR_ENT_ECART_GAGNANT']].fillna( df[['PAR_ENT_ECART_GAGNANT']].mean())
    df[['PAR_ENT_RAPPORT_GAGNANT_M']] = df[['PAR_ENT_RAPPORT_GAGNANT_M']].fillna( df[['PAR_ENT_RAPPORT_GAGNANT_M']].mean())
    df[['PAR_ENT_REU_PLACE']] = df[['PAR_ENT_REU_PLACE']].fillna( df[['PAR_ENT_REU_PLACE']].mean())
    df[['PAR_ENT_REUSSITE_GAGNE']] = df[['PAR_ENT_REUSSITE_GAGNE']].fillna( df[['PAR_ENT_REUSSITE_GAGNE']].mean())
    df[['PAR_ENT_VICTOIRE']] = df[['PAR_ENT_VICTOIRE']].fillna( df[['PAR_ENT_VICTOIRE']].mean())
    df[['PAR_ENT_NB_COURSE']] = df[['PAR_ENT_NB_COURSE']].fillna( df[['PAR_ENT_NB_COURSE']].mean())
    
    df[['pAR_JOC_ECART_GAGNANT']] = df[['pAR_JOC_ECART_GAGNANT']].fillna( df[['pAR_JOC_ECART_GAGNANT']].mean())
    df[['pAR_JOC_RAPPORT_GAGNANT_M']] = df[['pAR_JOC_RAPPORT_GAGNANT_M']].fillna( df[['pAR_JOC_RAPPORT_GAGNANT_M']].mean())
    df[['PAR_JOC_REU_PLACE']] = df[['PAR_JOC_REU_PLACE']].fillna( df[['PAR_JOC_REU_PLACE']].mean())
    df[['pAR_JOC_REUSSITE_GAGNE']] = df[['pAR_JOC_REUSSITE_GAGNE']].fillna( df[['pAR_JOC_REUSSITE_GAGNE']].mean())
    df[['pAR_JOC_VICTOIRE']] = df[['pAR_JOC_VICTOIRE']].fillna( df[['pAR_JOC_VICTOIRE']].mean())
    df[['PAR_JOC_ECART_PLACE']] = df[['PAR_JOC_ECART_PLACE']].fillna( df[['PAR_JOC_ECART_PLACE']].mean())
    df[['PAR_JOC_NB_COURSE']] = df[['PAR_JOC_NB_COURSE']].fillna( df[['PAR_JOC_NB_COURSE']].mean())
    df[['PAR_JOC_PLACE_3P']] = df[['PAR_JOC_PLACE_3P']].fillna( df[['PAR_JOC_PLACE_3P']].mean())
    df['PAR_RUESSITE_PLACE'] = df['PAR_RUESSITE_PLACE'].fillna( df['PAR_RUESSITE_PLACE'].mean())
    df[['PAR_REUSSITE_GAGNE']] = df[['PAR_REUSSITE_GAGNE']].fillna( 0)
    df[['PAR_REUSSITE_3P']] = df[['PAR_REUSSITE_3P']].fillna( df[['PAR_REUSSITE_3P']].mean())
    df[['PAR_CARRIERE']] = df[['PAR_CARRIERE']].fillna( df[['PAR_CARRIERE']].mean())
    df[['PAR_CARRIERE_Q']] = df[['PAR_CARRIERE_Q']].fillna( df[['PAR_CARRIERE_Q']].mean())
    df[['PAR_GAIN']] = df[['PAR_GAIN']].fillna( df[['PAR_GAIN']].mean())    
    df[['PAR_PLACE']] = df[['PAR_PLACE']].fillna( df[['PAR_PLACE']].mean())
    df[['PAR_PLACE_Q']] = df[['PAR_PLACE_Q']].fillna( df[['PAR_PLACE_Q']].mean())
    df[['PAR_CLASSE_AGE']] = df[['PAR_CLASSE_AGE']].fillna( df[['PAR_CLASSE_AGE']].mean())
        
    return df


In [121]:
index_col = ['IDCOURSE','IDPARTCIPANT']
start_time=timer()
df = lecture_data('d:\data_diabolo.csv', xnames, xindex_col=index_col, allure=allure_etudier, avec_index=False)
df2=df.copy()
#df.set_index("IDPARTCIPANT", inplace=True)
df=transformation(df,allure_etudier)
df=nettoyer_Nan(df)
print(timer(start_time))

2018-06-28 07:03:57.430890
DATA ....
2018-06-28 07:03:57.430890

 Time taken: 0 hours 0 minutes and 2.42 seconds.
None


----------- TRANSFORMATION --------- AJOUT DE COLONNE------------

2018-06-28 07:03:59.904254

 Time taken: 0 hours 0 minutes and 0.48 seconds.

 Time taken: 0 hours 0 minutes and 3.09 seconds.
None


In [122]:
df

,CO_DISTANCE,CO_PRIX,HIPPO,IDCOURSE,IDPARTCIPANT,PAR_AGE,PAR_CARRIERE,PAR_CARRIERE_Q,PAR_CLASSE_AGE,PAR_COTEDER,...,MUSIC_CHEVAL,MUSIC_ENT,MUSIC_JOC,PAR_VALEUR,PAR_ENT_ECART_PLACE,PAR_VICTOIRE,PAR_VICTOIRE_Q,PAR_ENT_NB_COURSE,FIN_ligne,SELECTION2
13,2,52000,1,1616,19849,6,41,1,36220,7,...,NaN,DADA2A1A,5MDA0M3A,0,2,7,1,808,FINFIN,0
14,2,52000,1,1616,19848,7,61,9,30670,199,...,NaN,5A3A7A6A,3A7A7A2A,0,1,5,0,192,FINFIN,0
15,2,52000,1,1616,19847,8,83,9,26118,60,...,NaN,DMDA0A4M,7A1ADADA,0,4,11,1,119,FINFIN,0
16,2,52000,1,1616,19846,6,45,0,33798,3,...,NaN,3A2M3A3A,4ADA2ADA,0,0,10,0,175,FINFIN,1
17,2,52000,1,1616,19845,7,64,4,27212,35,...,NaN,DADA2ADA,DADA2ADA,0,2,9,0,280,FINFIN,1
18,2,52000,1,1616,19844,6,44,2,30805,52,...,NaN,7ADA0M0A,DM3A6M0A,0,6,3,1,79,FINFIN,0
19,2,52000,1,1616,19843,8,56,3,21903,22,...,NaN,7A5A7A2M,DA0A1A0A,0,3,5,0,260,FINFIN,0
20,2,52000,1,1616,19842,6,42,3,26006,12,...,NaN,7A5A7A2M,2A0M1A1A,0,3,6,0,260,FINFIN,0
21,1,52000,1,1616,19841,7,37,1,16302,8,...,NaN,DA7M7A7A,DA3ADA2A,0,5,9,1,162,FINFIN,0
22,1,52000,1,1616,19840,8,63,1,14001,82,...,NaN,DA7ADA7A,1ADA3ADA,0,4,11,0,126,FINFIN,0


In [125]:
import math
def relation_1(a,b):
    if b==0:
        return 0
    else:
        return (a/b)*20
    
#---------------------------------------------------------------------------------------------------------------------

    
#---------------------------------------------------------------------------------------------------------------------    
def calcul_note(df, num_course, cri):
    valeur_critere=df.loc[df['IDCOURSE']==num_course,cri]  # selection critere    
    maxx=valeur_critere.max() #MAXI
    if ((maxx[0])>0):
        df.loc[df['IDCOURSE']==num_course,cri]  = ((df.loc[df['IDCOURSE']==num_course,cri])/maxx[0])*20
    else:
        df.loc[df['IDCOURSE']==num_course,cri]=0
    
    
    #print(   df.loc[df['IDCOURSE']==num_course,cri] )
    
        

    return  df

In [126]:
i=0
start_time=timer()
for n  in df.groupby(['IDCOURSE'],axis=0):
    nc=n[0]
    if math.fmod(i,1000)==0 or i<20:
        print("    print(N° ",i,timer(start_time))
    #df= calcul_note(df, nc, ['PAR_CARRIERE'] )
    df= calcul_note(df, nc, ['PAR_ENT_ECART_GAGNANT'] )
    df= calcul_note(df, nc, ['PAR_ENT_RAPPORT_GAGNANT_M'] )
    df= calcul_note(df, nc, ['PAR_ENT_REU_PLACE'] )
    df= calcul_note(df, nc, ['PAR_ENT_REUSSITE_GAGNE'] )
    df= calcul_note(df, nc, ['PAR_ENT_VICTOIRE'] ) 
    df= calcul_note(df, nc, ['PAR_ENT_NB_COURSE'] )   
    
    df= calcul_note(df, nc, ['pAR_JOC_ECART_GAGNANT'] )
    df= calcul_note(df, nc, ['pAR_JOC_RAPPORT_GAGNANT_M'] )
    df= calcul_note(df, nc, ['PAR_JOC_REU_PLACE'] )
    
    df= calcul_note(df, nc, ['pAR_JOC_REUSSITE_GAGNE'] )
    df= calcul_note(df, nc, ['pAR_JOC_VICTOIRE'] )    
    df= calcul_note(df, nc, ['PAR_JOC_ECART_PLACE'] )
    df= calcul_note(df, nc, ['PAR_JOC_NB_COURSE'] )    
    
    df= calcul_note(df, nc, ['PAR_JOC_PLACE_3P'] )    
    
    df= calcul_note(df, nc, ['PAR_REUSSITE_GAGNE'] )
    df= calcul_note(df, nc, ['PAR_RUESSITE_PLACE'] )
    df= calcul_note(df, nc, ['PAR_REUSSITE_3P'] )    
    df= calcul_note(df, nc, ['PAR_CARRIERE'] )
    df= calcul_note(df, nc, ['PAR_CARRIERE_Q'] )
    df= calcul_note(df, nc, ['PAR_GAIN'] )
    df= calcul_note(df, nc, ['PAR_PLACE'] )
    df= calcul_note(df, nc, ['PAR_PLACE_Q'] )
    df= calcul_note(df, nc, ['PAR_CLASSE_AGE'] )
    
    
    #df= calcul_note(df, nc, ['POIDS'] )              
    i=i+1
    




    

2018-06-28 07:04:34.340568

 Time taken: 0 hours 0 minutes and 0.07 seconds.
    print(N°  0 None

 Time taken: 0 hours 0 minutes and 0.85 seconds.
    print(N°  1 None

 Time taken: 0 hours 0 minutes and 1.06 seconds.
    print(N°  2 None

 Time taken: 0 hours 0 minutes and 1.23 seconds.
    print(N°  3 None

 Time taken: 0 hours 0 minutes and 1.4 seconds.
    print(N°  4 None

 Time taken: 0 hours 0 minutes and 1.56 seconds.
    print(N°  5 None

 Time taken: 0 hours 0 minutes and 1.72 seconds.
    print(N°  6 None

 Time taken: 0 hours 0 minutes and 1.87 seconds.
    print(N°  7 None

 Time taken: 0 hours 0 minutes and 2.03 seconds.
    print(N°  8 None

 Time taken: 0 hours 0 minutes and 2.18 seconds.
    print(N°  9 None

 Time taken: 0 hours 0 minutes and 2.34 seconds.
    print(N°  10 None

 Time taken: 0 hours 0 minutes and 2.49 seconds.
    print(N°  11 None

 Time taken: 0 hours 0 minutes and 2.64 seconds.
    print(N°  12 None

 Time taken: 0 hours 0 minutes and 2.81 seconds

In [127]:
df

,CO_DISTANCE,CO_PRIX,HIPPO,IDCOURSE,IDPARTCIPANT,PAR_AGE,PAR_CARRIERE,PAR_CARRIERE_Q,PAR_CLASSE_AGE,PAR_COTEDER,...,MUSIC_CHEVAL,MUSIC_ENT,MUSIC_JOC,PAR_VALEUR,PAR_ENT_ECART_PLACE,PAR_VICTOIRE,PAR_VICTOIRE_Q,PAR_ENT_NB_COURSE,FIN_ligne,SELECTION2
13,2,52000,1,1616,19849,6,9.534884,2.222222,36220,7,...,NaN,DADA2A1A,5MDA0M3A,0,2,7,1,808,FINFIN,0
14,2,52000,1,1616,19848,7,14.186047,20.000000,30670,199,...,NaN,5A3A7A6A,3A7A7A2A,0,1,5,0,192,FINFIN,0
15,2,52000,1,1616,19847,8,19.302326,20.000000,26118,60,...,NaN,DMDA0A4M,7A1ADADA,0,4,11,1,119,FINFIN,0
16,2,52000,1,1616,19846,6,10.465116,0.000000,33798,3,...,NaN,3A2M3A3A,4ADA2ADA,0,0,10,0,175,FINFIN,1
17,2,52000,1,1616,19845,7,14.883721,8.888889,27212,35,...,NaN,DADA2ADA,DADA2ADA,0,2,9,0,280,FINFIN,1
18,2,52000,1,1616,19844,6,10.232558,4.444444,30805,52,...,NaN,7ADA0M0A,DM3A6M0A,0,6,3,1,79,FINFIN,0
19,2,52000,1,1616,19843,8,13.023256,6.666667,21903,22,...,NaN,7A5A7A2M,DA0A1A0A,0,3,5,0,260,FINFIN,0
20,2,52000,1,1616,19842,6,9.767442,6.666667,26006,12,...,NaN,7A5A7A2M,2A0M1A1A,0,3,6,0,260,FINFIN,0
21,1,52000,1,1616,19841,7,8.604651,2.222222,16302,8,...,NaN,DA7M7A7A,DA3ADA2A,0,5,9,1,162,FINFIN,0
22,1,52000,1,1616,19840,8,14.651163,2.222222,14001,82,...,NaN,DA7ADA7A,1ADA3ADA,0,4,11,0,126,FINFIN,0


In [129]:
if allure_etudier==1:
    df.to_csv('d:\diabolo_trans_trot.csv', header=False,index=False)
if allure_etudier==2:
    df.to_csv('d:\diabolo_trans_galop.csv', header=False,index=False)    
    


KeyError: "['PAR_ARRIVEE'] not in index"